In [ ]:
'''
마스크 착용하면 못 찾음 실패
'''

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import winsound

In [2]:
model_path = './98_LearningData/01_Example/'

In [3]:
face_cascades = cv2.CascadeClassifier('./98_LearningData/00_haarcascades/haarcascade_frontalface_default.xml')
eye_cascades = cv2.CascadeClassifier('./98_LearningData/00_haarcascades/haarcascade_lefteye_2splits.xml')

In [7]:
def face_detector(img, size = 0.5):
    #이미지를 흑백으로 변경
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #얼굴 검출
    faces = face_cascades.detectMultiScale(grayscale_img, scaleFactor=1.1, minNeighbors=5, minSize=(100,100), flags=cv2.CASCADE_SCALE_IMAGE)
    
    if faces is():
        return img,[], 0
    
    iFaceCnt = 0
    for(x,y,w,h) in faces:
        #눈을 추려내기 위해서 이미지 크롭
        img_trim = img[y:y+h, x:x+w]
        
        #눈 추려내기
        eye_list = eye_cascades.detectMultiScale(img_trim,scaleFactor = 1.15, minNeighbors=1,minSize=(10,10))
        
        #눈이 있으면서 2개 이상)
        if len(eye_list) >= 2:
            #얼굴에 네모 표기하기
            cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,255),2)

            #눈에 네모 표기 하기(눈에표기하면 인식 안됨 네모도 얼굴의 일부로 인식)
            #for eye in eye_list:
                #eyeX,eyeY,eyeW,eyeH = eye
                #cv2.rectangle(img, (eyeX + x, eyeY + y), (eyeX + eyeW + x, eyeY + eyeH + y), (0, 255, 0), thickness=8)

            iFaceCnt += 1
            roi = img[y:y+h, x:x+w]
            roi = cv2.resize(roi, (200,200))
        else:
            return img,[], 0
    return img, roi, iFaceCnt   #검출된 좌표에 사각 박스 그리고(img), 검출된 부위를 잘라(roi) 전달

In [11]:
# model : 마스크 검출 모델
model = load_model(model_path + 'mask_detector.model')

In [14]:
#카메라 열기 
cap = cv2.VideoCapture(0)
print('width :%d, height : %d' % (cap.get(3), cap.get(4)))
while True:
    #카메라로 부터 사진 한장 읽기 
    ret, frame = cap.read()
    # 얼굴 검출 시도 
    image, face, iFaceCnt = face_detector(frame)
    try:        
        # 추출한 얼굴영역을 전처리
        face_input = cv2.resize(face, dsize=(224, 224))
        face_input = cv2.cvtColor(face_input, cv2.COLOR_BGR2RGB)
        face_input = preprocess_input(face_input)
        face_input = np.expand_dims(face_input, axis=0)
        
        # 마스크 검출 모델로 결과값 return
        mask, nomask = model.predict(face_input).squeeze()

        # 마스크를 꼈는지 안겼는지에 따라 라벨링해줌
        if mask > nomask:
            color = (0, 255, 0)
            label = 'Mask %d%%' % (mask * 100)
        else:
            color = (0, 0, 255)
            label = 'No Mask %d%%' % (nomask * 100)
            frequency = 2500  # Set Frequency To 2500 Hertz
            duration = 1000  # Set Duration To 1000 ms == 1 second
         
        # 화면에 얼굴부분과 마스크 유무를 출력해해줌
        #cv2.rectangle(image, pt1=(x1, y1), pt2=(x2, y2), thickness=2, color=color, lineType=cv2.LINE_AA)
        #cv2.putText(image, text=label, org=(x1, y1 - 10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8,
                    #color=color, thickness=2, lineType=cv2.LINE_AA)
        
        
        cv2.putText(image, str(iFaceCnt) + " Face Found " + label, (100, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)    
        cv2.imshow('Face Cropper', image)
    except:
        #얼굴 검출 안됨 
        cv2.putText(image, "Face Not Found", (200, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('Face Cropper', image)
        pass
    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()

width :640, height : 480
